# MINE 4213 - Proyecto parte 1
Elaborado por Juan Manuel Rivera

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

## 1. Carga de datos

### 1.1 Datos de proyectos de inversión
Los datos tienen la siguiente estructura:

    +---bpin
    |   *.csv.gz 

In [0]:
proyectos_inversion = (spark
      .read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv("wasbs://sid@uniandesyjt.blob.core.windows.net/bpin")
      )

proyectos_inversion.printSchema()

In [0]:
proyectos_inversion.show()

In [0]:
display(proyectos_inversion)

In [0]:
proyectos_inversion.count()

### 1.2 Datos contratos electrónicos
Los datos tienen la siguiente estructura:

    +---anno_firma=2015 
    |       
    *.json.gz 
    | 
    +---anno_firma=2016 
    |       
    *. json.gz 
    | 
    +---anno_firma=2017 
    |       
    *. json.gz  
    | 
    +--- … 

In [0]:
secop = (spark
      .read
      .json("wasbs://sid@uniandesyjt.blob.core.windows.net/secop")
      )

In [0]:
secop.printSchema()

In [0]:
display(secop)

In [0]:
secop.count()

## 2. Top 10 proveedores con mayor valor de contratos en el 2024
Identifique los 10 proveedores que han tenido el mayor valor de contratos durante el año 2024

Se aprovechará que los datos del SECOP están particionados por el año de la firma para filtrarlos al momento de leerlos.

In [0]:
# Se selecciona un sub cojnunto con las columnas a utilizar
secop_2024 = (
  secop
    .filter(secop.anno_firma == 2024)
    .select("documento_proveedor", "proveedor_adjudicado", "valor_del_contrato")
    .groupBy("documento_proveedor", "proveedor_adjudicado")
    .agg(f.sum("valor_del_contrato")
    .alias("valor_total_de_contratos"))
    .orderBy(f.desc("valor_total_de_contratos"))
    .limit(10)
  )
display(secop_2024)

## 3. Top 10 proyectos con mayor valor por pagar
Identifique los 10 Proyectos de inversión que han tenido el mayor valor de contratos sin pagar (valor del contrato menos el valor pagado). La respuesta debe incluir el nombre de los proyectos y el valor sin pagar.

In [0]:
proyectos_valor_por_pagar = (
    proyectos_inversion
        .withColumn(
            "ValorPorPagar",
            f.round(proyectos_inversion["ValorTotalProyecto"] - proyectos_inversion["ValorPagoProyecto"], 2)
            )
        .select("NombreProyecto", "ValorPorPagar")
)

top_10_valor_por_pagar = (
    proyectos_valor_por_pagar
    .orderBy(f.desc("ValorPorPagar"))
    .limit(10)
    )
display(top_10_valor_por_pagar)

## 4. Top 5 proveedores con mayor valor de contratos cada año
Identifique para cada año el top 5 de los proveedores con mayor valor de contratos. La respuesta debe incluir el nombre de los proveedores, el valor de sus contratos y su posición en el top.

Crearemos una ventana para evaluar el valor total del contrato por año

In [0]:
ventas_por_anno = (
    secop
        .groupBy("proveedor_adjudicado", "anno_firma")
        .agg(f.sum("valor_del_contrato").alias("valor_total_anno"))
    )

In [0]:
ventana_por_anno = (
    Window
    .partitionBy("anno_firma")
    .orderBy(f.desc("valor_total_anno"))
    )

In [0]:
top_proveedores_por_anno = (
  ventas_por_anno
    .withColumn("rank", f.row_number().over(ventana_por_anno))
    .filter(f.col("rank") <= 5)
  )
display(top_proveedores_por_anno)

## 5. Proveedores que firmaron contratos en 2024 pero no en 2020
Identifique el número de proveedores que firmaron contratos en el 2024 y NO firmaron 
contratos en el 2020.

In [0]:
proveedores_2024 = (
    secop.
        filter(secop.anno_firma == 2024)
        .select("documento_proveedor")
        .distinct()
)
proveedores_2020 = (
    secop.
        filter(secop.anno_firma == 2020)
        .select("documento_proveedor")
        .distinct()
)

In [0]:
proveedores_solo_2024 = (
    proveedores_2024
    .join(proveedores_2020, on="documento_proveedor", how="leftanti")
    .agg(f.count("documento_proveedor").alias("proveedores_solo_2024"))
)

display(proveedores_solo_2024)

## 6. Media y mediana de los contratos por "Prestación de servicios"
Calcule el valor promedio de los contratos de “Prestación de servicios” y una aproximación de la mediana (el percentil 50). Construya esta respuesta de manera que se minimice el tiempo de cálculo, aunque se sacrifique precisión.

In [0]:
# Primero veremos los datos únicos de la columna tipo de contrato
display(secop.select(f.col("tipo_de_contrato")).distinct())

In [0]:
# Se mostrará la media de valor del tipo de contrato
display(
    secop
        .filter(f.col("tipo_de_contrato") == "Prestación de servicios")
        .select(f.avg("valor_del_contrato").alias("Valor_promedio_prestacion_servicios"))
)

In [0]:
# Se calculará la mediana aproximada
# Se mostrará la mediana para los contratos de tipo prestación de servicio, con un error del 5%
# Exacto median 18702456
display(
    secop
        .filter(f.col("tipo_de_contrato") == "Prestación de servicios")
        .approxQuantile("valor_del_contrato", [0.5], 0.05))



La diferencia marcada entre la mediana y la media sugiere que la distribución podría estar sesgada hacia valores de contrato más bajos. Otra posible explicación sería que hay algunos contratos con valores excepcionalmente altos, lo cuál afecta el promedio.

## 7. Año con mayor número de proveedores distintos
Cuál fue el año en el que firmaron contratos un mayor número de proveedores distintos. Construya esta respuesta de manera que se minimice el tiempo de cálculo, aunque se sacrifique precisión.

In [0]:
# Se usará la función de conteo aproximado
display(
  secop
    .groupBy("anno_firma")
    .agg(f.approx_count_distinct("documento_proveedor").alias("num_proveedores"))
    .orderBy("num_proveedores", ascending=False)
)

## 8. Las 20 palabras más comunes en los objetos contractuales
Cuáles son las 20 palabras más comunes en los objetos contractuales del 20% de los contratos más altos del 2020. Para esta respuesta omita stopwords (https://en.wikipedia.org/wiki/Stop_word, https://pypi.org/project/stop-words/), puntuaciones y no diferencie entre mayúsculas y minúsculas.

In [0]:
# Se tomó la lista de stopwords de la librería NLTK (Natural Language Toolkit).
stopwords = ['fuéramos', 'sí', 'fuésemos', 'yo', 'hayas', 'tenías', 'habrán', 'habían', 'suya', 'estamos', 'tiene', 'al', 'hubisteis', 'tendremos', 'os', 'estás', 'hubiese', 'habríamos', 'estoy', 'estarás', 'hay', 'tuyas', 'contra', 'de', 'tuvieses', 'estáis', 'habéis', 'habremos', 'esa', 'nuestras', 'fueseis', 'tendrían', 'estada', 'esos', 'estaban', 'estuve', 'habrían', 'sentido', 'más', 'eran', 'ti', 'estad', 'hubimos', 'esté', 'tienen', 'tenían', 'estaría', 'hubierais', 'mía', 'sois', 'estuvieseis', 'te', 'qué', 'tengas', 'tendríamos', 'eres', 'para', 'algunas', 'han', 'has', 'estaréis', 'nuestra', 'las', 'habríais', 'serías', 'fuisteis', 'estado', 'como', 'tengáis', 'será', 'sobre', 'ese', 'una', 'hubo', 'otro', 'estuvieron', 'fuera', 'estabais', 'estuviesen', 'esto', 'hubieseis', 'ni', 'vuestras', 'habidos', 'estadas', 'sus', 'hubiste', 'vuestros', 'estuvieses', 'seríais', 'estaríamos', 'nos', 'tú', 'estaríais', 'donde', 'estemos', 'hubiésemos', 'tuvieseis', 'del', 'estaremos', 'teníamos', 'estuvo', 'sentid', 'suyas', 'entre', 'eras', 'tuvierais', 'hayan', 'tuviéramos', 'también', 'he', 'tuviésemos', 'estuviéramos', 'tenido', 'y', 'tendría', 'otra', 'me', 'haya', 'vuestro', 'es', 'hube', 'e', 'tuviera', 'vuestra', 'o', 'el', 'habrás', 'tendrías', 'tenida', 'somos', 'fui', 'hubieses', 'estés', 'su', 'fuese', 'tendrás', 'nosotros', 'tenéis', 'antes', 'ellos', 'estaba', 'estarán', 'seas', 'estar', 'fue', 'tendré', 'tanto', 'se', 'estuviera', 'serán', 'habido', 'tuya', 'tendrá', 'seáis', 'sean', 'otros', 'con', 'fuerais', 'seréis', 'fueron', 'tendréis', 'hubiesen', 'estuvierais', 'teníais', 'todo', 'tienes', 'hayamos', 'no', 'fuimos', 'habida', 'ante', 'sentidos', 'tendrán', 'los', 'tuviste', 'estas', 'fueran', 'algunos', 'le', 'esas', 'a', 'tus', 'hubieran', 'míos', 'serían', 'estaré', 'hemos', 'tuvisteis', 'estados', 'sería', 'sin', 'cuando', 'erais', 'habíamos', 'sentida', 'tuyos', 'tengan', 'la', 'mis', 'tenidas', 'nuestros', 'tuvieran', 'era', 'durante', 'habías', 'vosotros', 'estarían', 'hasta', 'estén', 'ha', 'tuvieras', 'serás', 'tenemos', 'son', 'quien', 'estuviese', 'estabas', 'vosotras', 'hubieron', 'mucho', 'estarías', 'por', 'tuve', 'poco', 'estéis', 'tendríais', 'tenidos', 'soy', 'quienes', 'muy', 'había', 'tuviesen', 'habré', 'habrá', 'unos', 'habrías', 'tuvimos', 'nosotras', 'suyo', 'ya', 'muchos', 'estuviésemos', 'estos', 'tuyo', 'todos', 'está', 'hayáis', 'ella', 'siente', 'mías', 'estará', 'tu', 'tenga', 'tuvieron', 'eso', 'fueras', 'seremos', 'hubieras', 'estuvieras', 'estuviste', 'tened', 'seamos', 'tuvo', 'nada', 'habidas', 'seríamos', 'sentidas', 'tenía', 'están', 'estuvieran', 'estábamos', 'un', 'suyos', 'tuviese', 'otras', 'nuestro', 'que', 'tengamos', 'habréis', 'mío', 'mi', 'hubiéramos', 'habíais', 'pero', 'ellas', 'en', 'uno', 'estuvimos', 'algo', 'les', 'cual', 'estuvisteis', 'estando', 'porque', 'desde', 'habría', 'esta', 'hubiera', 'tengo', 'teniendo', 'fuiste', 'sea', 'mí', 'él', 'fuesen', 'seré', 'lo', 'habiendo', 'este', 'éramos', 'sintiendo', 'fueses']

In [0]:
# Se calculará el valor aproximado de corte para el valor del contrato
q_80 = (
    secop
        .filter(f.col("anno_firma") == 2020)
        .approxQuantile("valor_del_contrato", [0.8], 0.01)
        )
display(q_80)

In [0]:
top_objetos_2020 = (
    secop
        .filter(
            (secop.anno_firma == 2020) & 
            (secop.valor_del_contrato >= q_80[0]))
        .select("objeto_del_contrato")
                      )
display(top_objetos_2020)

In [0]:
top_objetos_2020 = top_objetos_2020.withColumn(
    "word",
    f.explode(f.split(f.col("objeto_del_contrato"), r"[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ]+"))
)

display(top_objetos_2020)

In [0]:
top_objetos_2020=top_objetos_2020.withColumn("word", f.lower(f.col("word")))
top_objetos_2020 = top_objetos_2020.filter(f.col("word") != "")
top_objetos_2020 = top_objetos_2020.filter((f.length("word") > 1) & (~f.col("word").isin(stopwords) ))
display(top_objetos_2020)

In [0]:
# Count frequencies
word_counts = (
    top_objetos_2020.groupBy("word")
    .count()
    .orderBy(f.desc("count"))
)

display(word_counts)